#  USC EE559 Final Project Spring 2021
#  Online News Popularity Prediction
## Mingjian Shi 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor 
import pickle
from sklearn.pipeline import Pipeline

# Performance Measure

In [2]:
#0. Performance Measure
#Trivial system
def self_MSE(y):
    y_m = np.mean(y)
    MSE = 0
    for i in range(np.size(y)):
        MSE = MSE + (y[i] - y_m)**2
    MSE = MSE/np.size(y)
    return MSE


#Calculate p_MSE
#def p_MSE(y_train,y_predict):
    #pMSE = 0
    #r = 10
    #for i in range(np.size(y_train)):
        #pMSE = pMSE + ((y_train[i] - y_predict[i]) / (r + y_train[i]))**2
    #pMSE = pMSE / np.size(y_train)
    #return pMSE

# Under the help of our TA, we changed p_MSE to the following
def p_MSE(y, y_pred):
    diff = (y - y_pred)/(y+10)
    pMSE = np.mean(np.power(diff, 2))
    return pMSE


#Calculate p_MAE
#def p_MAE(y_train, y_predict):
    #pMAE = 0
    #r = 10
    #for i in range(np.size(y_train)):
        #pMAE = pMAE + np.abs((y_train[i] - y_predict[i]) / (y_train[i] + r))
    #pMAE = pMAE / np.size(y_train)
    #return pMAE
    
# Under the help of our TA, we changed p_MAE to the following    
def p_MAE(y, y_pred):
    diff = (y - y_pred)/(y+10)
    pMAE = np.mean(np.abs(diff))
    return pMAE

#Calculate R_squared 
def R_squared(y_train, y_predict):
    y_mse = self_MSE(y_train)
    y_pre_mse = mean_squared_error(y_train,y_predict)
    rSquared = 1 - y_pre_mse/y_mse
    return rSquared
#Calculate Modified R_squared
def modified_R_squared(y_train, y_predict,pMSE):
    m_R = 0
    r = 10
    modified = 0
    y_mean = np.mean(y_train)
    for i in range(np.size(y_train)):
        modified = modified + ((y_train[i] - y_mean) / (r + y_train[i]))**2
    modified = modified / np.size(y_train) 
    m_R = 1 - (pMSE / modified)
    return m_R






# 0. Load Data and Downsampling

In [3]:
#---------------------------------------------------------------------------
#1. Load Data
#Training set
orig_train = 'NEWS_Training_data.csv'
orig_label = 'NEWS_Training_label.csv'
df_orig_train_data = pd.read_csv(orig_train, header = 0)
df_orig_label_data = pd.read_csv(orig_label, header = 0)

#Test set
test_data = 'NEWS_Test_data.csv'
test_label = 'NEWS_Test_label.csv'
X_orig_test = pd.read_csv(test_data, header = 0)
Y_orig_test = pd.read_csv(test_label, header = 0)


#---------------------------------------------------------------------------
#Down Sampling 
df_data = pd.concat([df_orig_train_data,df_orig_label_data],axis=1)

down_sample = df_data.sample(frac = 0.1)
down_sample.shape

#Split label and data
df_orig_train = down_sample.drop(['shares'],axis =1)
df_orig_label = down_sample['shares']

print('Entire Training Data Set: ',np.shape(df_orig_train_data),'\n')
print('Data after Down-Sampling: ',np.shape(df_orig_train),'\n')

print('Entire Test Data Set:', np.shape(X_orig_test))
#---------------------------------------------------------------------------

Entire Training Data Set:  (35644, 58) 

Data after Down-Sampling:  (3564, 58) 

Entire Test Data Set: (2000, 58)


# 1.1 Remove Outliers
## (We didn't use it eventually)

In [ ]:
#Remove Outlier

#First, lets take a look at the labels distribution
df_orig_label_data.hist(bins = 50, figsize = (10,10))
plt.title('Original shares data')
plt.show()

#Let's take a look at the shares<10000
df_label = df_orig_label_data[df_orig_label_data<10000]
df_label.hist(bins = 50, figsize = (10,10))
plt.title('Shares data for #shares<10000')
plt.show()
print('Median:', np.median(df_orig_label_data),'\n')
print('Mean:', np.mean(df_orig_label_data),'\n')
print('Std:', np.std(df_orig_label_data),'\n')
print('(Median+2std):',np.median(df_orig_label_data) + 2*np.std(df_orig_label_data),'\n') 
print('(median-2std):',np.mean(df_orig_label_data) - 2*np.std(df_orig_label_data),'\n')
print('Data size for #shares<10000\n',df_label[df_label['shares']<10000].shape)
#We found out using Median can better represent the data based on #shares
#We narrow our data range from 0 to (median + 2*std) 
m = np.median(df_orig_label_data)
std = np.std(df_orig_label_data)
drop_limit = m + 2*std
df_data = pd.concat([df_orig_train_data,df_orig_label_data],axis=1)
df_data = df_data[df_data['shares'] < int(drop_limit)]

df_orig_train = df_data.drop(['shares'],axis =1)
df_orig_label = df_data['shares']
df_orig_label.hist(bins = 50, figsize = (10,10))
plt.title('Narrowed Shares data (median + 2*std)')
plt.show()

#Compare data size After outliers were removed
print('Data size before remove outlier',np.shape(df_orig_train_data),'\n')
print('Data size after remove outlier',np.shape(df_orig_train),'\n')

# 1.2 Combine Features and Feature Engineering 
## (We didn't Combine any features eventually)

In [ ]:
#Combine Feature
df_train = df_orig_train

#There are features can be combined into one new feature
#Some thoughts are from https://towardsdatascience.com/using-columntransformer-to-combine-data-processing-steps-af383f7d5260
def FeaturesCombine(input_data, original_f, combined_f):
    
    index = 0
    input_data[combined_f] = index
    
    for feature_name in original_f:
        index += 1
        input_data.loc[input_data[feature_name] == 1, combined_f] = index
        input_data = input_data.drop([feature_name], axis = 1)
        
    return input_data

#We found the weekdays category can be merged to a new features called weekdays with values from 1-7 to represent Mon-Sun
days = ['weekday_is_monday','weekday_is_tuesday','weekday_is_wednesday','weekday_is_thursday','weekday_is_friday','weekday_is_saturday','weekday_is_sunday']
df_train = FeaturesCombine(df_train, days, 'weekdays')

#We found the data_channles can be merged to a new features called data_channels with 1-6 represent lifestyle to world catagory
channels = ['data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 'data_channel_is_socmed','data_channel_is_tech','data_channel_is_world'] 
df_train = FeaturesCombine(df_train, channels, 'data_channels')
np_train = np.array(df_train)
np_label = np.array(df_orig_label)

print('After FeatureCombine',np_train.shape,'\n')

# 2. Split Train and Val

In [4]:
#---------------------------------------------------------------------------

#Split Train and Val 

# 1.For DownSampled Data
X_train = np.array(df_orig_train)
Y_train = np.array(df_orig_label)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2, random_state =40)

print('1.Split into Training and Validation Set(DownSampled Data):\n')
print('X_train Size:',X_train.shape)
print('Y_train Size:',Y_train.shape)
print('X_Val Size:',X_val.shape)
print('Y_Val Size:',Y_val.shape,'\n')

# 2.For Entire Data Set 
df_data_all = pd.concat([df_orig_train_data,df_orig_label_data],axis=1)
df_train_all = df_data_all.drop(['shares'],axis =1)
df_label_all = df_data_all['shares']
X_train_all = np.array(df_train_all)
Y_train_all = np.array(df_label_all)
X_train_all, X_val_all, Y_train_all, Y_val_all = train_test_split(X_train_all, Y_train_all, test_size = 0.2, random_state =40)

print('2.Split into Training and Validation Set(Entire Data set):\n')
print('X_train_all Size:',X_train_all.shape)
print('Y_train_all Size:',Y_train_all.shape)
print('X_Val_all Size:',X_val_all.shape)
print('Y_Val_all Size:',Y_val_all.shape,'\n')

#--------------------------------------------------------------------------


1.Split into Training and Validation Set(DownSampled Data):

X_train Size: (2851, 58)
Y_train Size: (2851,)
X_Val Size: (713, 58)
Y_Val Size: (713,) 

2.Split into Training and Validation Set(Entire Data set):

X_train_all Size: (28515, 58)
Y_train_all Size: (28515,)
X_Val_all Size: (7129, 58)
Y_Val_all Size: (7129,) 



# 3.Normalize Data

In [5]:
#--------------------------------------------------------------------------

#Normalize Data

# 1.For DownSampled Data
scaler1 = StandardScaler()
scaler1.fit(X_train)
X_train = scaler1.transform(X_train)
X_val = scaler1.transform(X_val)

print('1.Normalization for DownSampled Data Completed......\n')

# 2.For Entire Data Set (Training)
scaler2 = StandardScaler()
scaler2.fit(X_train_all)
X_train_all = scaler1.transform(X_train_all)
X_val_all = scaler1.transform(X_val_all)
print('2.Normalization for Entire Data set Completed......\n')


#--------------------------------------------------------------------------

1.Normalization for DownSampled Data Completed......

2.Normalization for Entire Data set Completed......



# 4. Regression with different Model 

##  Trival System

In [6]:
# Trival System 
y_mse = self_MSE(df_orig_label_data.to_numpy())
print('Trival sys MSE:', y_mse)
print('\n')

Trival sys MSE: [1.40846919e+08]




## 4.0 Linear Regression 

In [7]:
#Load data
train_data = df_orig_train_data.to_numpy()
train_label = df_orig_label_data.to_numpy()
train_label = train_label[:,0]

X_test = X_orig_test.to_numpy()
Y_test = Y_orig_test.to_numpy()
Y_test = Y_test[:,0]

#Normalize Training and Test data
scaler_best = StandardScaler()
scaler_best.fit(train_data)
train_data = scaler_best.transform(train_data)
test_data = scaler_best.transform(X_test)


#Using Linear Regression to Training and test to report the base line model
reg_lin = LinearRegression().fit(train_data, train_label)
test_pred = reg_lin.predict(test_data)

#Performance Measure
MAE = mean_absolute_error(Y_test,test_pred)
pMSE = p_MSE(Y_test,test_pred)
pMAE = p_MAE(Y_test,test_pred)
rSquared = R_squared(Y_test,test_pred)
m_rSquared = modified_R_squared(Y_test, test_pred, pMSE)
print('Linear Regression Baseline Model')
print('MAE:',MAE)
print('pMSE:',pMSE)
print('pMAE:',pMAE)
print('R_squared:', rSquared)
print('Modified_R_Squared:',m_rSquared)

Linear Regression Baseline Model
MAE: 2798.4787747951978
pMSE: 6.93660065041956
pMAE: 1.5713788322876932
R_squared: 0.04423602205078592
Modified_R_Squared: 0.20344363931290588


## 4.1. SVR

In [7]:
#--------------------------------------------------------------------------
# SVR with 5 fold Cross validation 
def SVRCrossValidation(x_train, y_train):
    kf = KFold(n_splits=5)
    gamma_ = np.logspace(-3, 3, num=50, base=10.0)
    C_ = np.logspace(-3, 3, num=50, base=10.0)
    pMSE = np.zeros([np.size(gamma_), np.size(C_)], dtype=float)
    pMAE = np.zeros([np.size(gamma_), np.size(C_)], dtype=float)

    #For each gamma, try each C
    for Ng, g in enumerate(gamma_):
        for Nc, c in enumerate(C_):
            k = 0
            pmse_av = 0
            pmae_av = 0
            pmse = np.zeros([5, 1], dtype=float)
            pmae = np.zeros([5, 1], dtype=float)
            #split
            for train_index, val_index in kf.split(x_train, y_train):
                data_train, data_val = x_train[train_index], x_train[val_index]
                label_train, label_val = y_train[train_index], y_train[val_index]
                #Define the model
                reg = SVR(C=c, kernel='rbf', gamma=g)
                reg.fit(data_train, label_train)
                y_pred = reg.predict(data_val)
                pmse[k] = p_MSE(label_val, y_pred)
                pmae[k] = p_MAE(label_val, y_pred)
                k += 1
            pmse_av = np.mean(pmse)
            pmae_av = np.mean(pmae)
            pMSE[Ng,Nc] = pmse_av
            pMAE[Ng,Nc] = pmae_av
    return pMSE, pMAE

#SVR using validation and training(No Cross validation)
def SVRVal(x_train, y_train,x_val,y_val):
    gamma_ = np.logspace(-3, 3, num=50, base=10.0)
    C_ = np.logspace(-3, 3, num=50, base=10.0)
    pMSE = np.zeros([np.size(gamma_), np.size(C_)], dtype=float)
    pMAE = np.zeros([np.size(gamma_), np.size(C_)], dtype=float)
    
    for Ng, g in enumerate(gamma_):
        for Nc, c in enumerate(C_):
            #Define the model
            reg = SVR(C=c, kernel='rbf', gamma=g)
            reg.fit(x_train, y_train)
            y_pred = reg.predict(x_val)
            
            #Store the value in     
            pMSE[Ng,Nc] = p_MSE(y_val,y_pred)
            pMAE[Ng,Nc] = p_MAE(y_val,y_pred)
    return pMSE, pMAE

### 4.1.1 SVR Model Selection

In [ ]:
#Calculate the performance 
pMSE,pMAE = SVRVal(X_train, Y_train, X_val, Y_val)

gamma_ = np.logspace(-3, 3, num=50, base=10.0)
C_ = np.logspace(-3, 3, num=50, base=10.0)

#Find the best parameters based on MSE
# if there are more than one best MSE, choose the one with best MAE
def findBestPara_MSE(pMSE,pMAE):
    best_mse = np.min(pMSE)
    Ng_best,Nc_best = np.where(pMSE == best_mse)
    if np.size(Ng_best) == 1:
        return Ng_best,Nc_best
    else:# if the best is not the only one, we need to find based on MAE
        candidate_pmae = np.zeros([np.size(Ng_best),1],dtype = float)
        for i in range(np.size(Ng_best)):
            candidate_pmae[i] = pMAE[Ng_best[i],Nc_best[i]]
        best_mae = np.min(candidate_pmae)
        i,j = np.where(candidate_pmae == best_mae)
        return Ng_best[i[0]],Nc_best[i[0]]
#call the function 
Ng_best, Nc_best = findBestPara_MSE(pMSE,pMAE)

print('SVR Best Model(MSE Based):')
print('Best gamma:',gamma_[Ng_best])
print('Best C:',C_[Nc_best])
print('Best pMSE:',pMSE[Ng_best,Nc_best])
print('Best pMAE:',pMAE[Ng_best,Nc_best])
print('Best Ng',Ng_best)
print('Best Nc',Nc_best,'\n')


#Find the best parameters based on MAE
# if there are more than one best MAE, choose the one with best MSE
def findBestPara_MAE(pMSE,pMAE):
    best_mae = np.min(pMAE)
    Ng_best,Nc_best = np.where(pMAE == best_mae)
    if np.size(Ng_best) == 1:
        return Ng_best,Nc_best
    else:# if the best is not the only one, we need to find based on MSE
        candidate_pmse = np.zeros([np.size(Ng_best),1],dtype = float)
        for i in range(np.size(Ng_best)):
            candidate_pmse[i] = pMSE[Ng_best[i],Nc_best[i]]
        best_mse = np.min(candidate_pmse)
        i,j = np.where(candidate_pmse == best_mse)
        return Ng_best[i[0]],Nc_best[i[0]]
#call the function
Ng_best, Nc_best = findBestPara_MAE(pMSE,pMAE)

print('SVR Best Model(MAE Based):')
print('Best gamma:',gamma_[Ng_best])
print('Best C:',C_[Nc_best])
print('Best pMSE:',pMSE[Ng_best,Nc_best])
print('Best pMAE:',pMAE[Ng_best,Nc_best])
print('Best Ng',Ng_best)
print('Best Nc',Nc_best)

### 4.1.2 SVR Model 1:  [g,C] = [0.00719686, 138.94954944] (Validation Result)

In [10]:
#After the previous step 
#We find two best pairs for SVR

#1.[g,C] = [0.00719686, 138.94954944]
#2.[g,C] = [0.00719686, 79.06043211]

#Then we run on all the data 2 times for the 2 best pairs and compare:
gamma_ = np.logspace(-3, 3, num=50, base=10.0)
C_ = np.logspace(-3, 3, num=50, base=10.0)

#1st Run with [g,C] = [0.00719686, 138.94954944]
svr1 = SVR(C=C_[42], kernel='rbf', gamma=gamma_[7])
svr1.fit(X_train_all, Y_train_all)
Y_pred_all = svr1.predict(X_val_all)
pMSE = p_MSE(Y_val_all,Y_pred_all)
pMAE = p_MAE(Y_val_all,Y_pred_all)
print('SVR with [g,C] = [0.00719686, 138.94954944]:')
print('pMSE:',pMSE)
print('pMAE:',pMAE)

SVR with [g,C] = [0.00719686, 138.94954944]:
pMSE: 1.1536588181895415
pMAE: 0.5709751439138046


### 4.1.3 SVR Model 2: [g,C] = [0.00719686, 79.06043211] (Validation Result)

In [10]:
#2nd Run with [g,C] = [0.00719686, 79.06043211]

svr2 = SVR(C=C_[40], kernel='rbf', gamma=gamma_[7])
svr2.fit(X_train_all, Y_train_all)
Y_pred_all = svr2.predict(X_val_all)
pMSE = p_MSE(Y_val_all,Y_pred_all)
pMAE = p_MAE(Y_val_all,Y_pred_all)
rSquared = R_squared(Y_val_all,Y_pred_all)
print('2nd Run SVR with [g,C] = [0.00719686, 79.06043211]:')
print('pMSE:',pMSE)
print('pMAE:',pMAE)

2nd Run SVR with [g,C] = [0.00719686, 79.06043211]:
pMSE: 1.1389524902128147
pMAE: 0.5698028364365723


### 4.1.4 SVR Model selection Results:
#### Based on the 2 SVR Models, we find Model 2 is better! 
#### Model Parameters: [g,C] = [0.00719686, 79.06043211]
#### Validation Results: 
#### pMSE: 1.15148983507122
#### pMAE: 0.5700800579704007

### 4.1.5 Report Best SVR Model(Entire Training Dataset)

In [11]:
#Gamma and C
gamma_ = np.logspace(-3, 3, num=50, base=10.0)
C_ = np.logspace(-3, 3, num=50, base=10.0)

#Best C is the 40th C, best gamma is 7th
svr_best = SVR(C=C_[40], kernel='rbf', gamma=gamma_[7])
svr_best.fit(X_train_all, Y_train_all)
Y_pred_all = svr_best.predict(X_val_all)

#Performance Measure
pMSE = p_MSE(Y_val_all,Y_pred_all)
pMAE = p_MAE(Y_val_all,Y_pred_all)
rSquared = R_squared(Y_val_all,Y_pred_all)
m_rSquared = modified_R_squared(Y_val_all, Y_pred_all, pMSE)
MAE = mean_absolute_error(Y_val_all,Y_pred_all)
print('Best SVR Model\n [g,C] = [gamma_[7], C_[40]] = [0.00719686, 79.06043211]:\n')
print('MAE:',MAE)
print('pMSE:',pMSE)
print('pMAE:',pMAE)
print('R_squared:', rSquared)
print('Modified_R_Squared:',m_rSquared)
print(svr_best.score(X_val_all, Y_val_all))

Best SVR Model
 [g,C] = [gamma_[7], C_[40]] = [0.00719686, 79.06043211]:

MAE: 2226.0999205355274
pMSE: 1.1389524902128147
pMAE: 0.5698028364365723
R_squared: -0.015921993152419933
Modified_R_Squared: 0.8701497349230467
-0.015921993152438585


## 4.2. Non-Linear Regression with PolyFeatures

In [34]:
#With Cross Validation 
def polyCrossValidation(x_train, y_train, min_M, max_M):
    kf = KFold(n_splits=5)
    poly_order = np.linspace(min_M, max_M, num = max_M - min_M+1)
    pMSE = np.zeros([max_M - min_M +1, 1], dtype=float)
    pMAE = np.zeros([max_M - min_M +1, 1], dtype=float)
    
    for Nm, m in enumerate(poly_order):
        k = 0
        pmse_av = 0
        pmae_av = 0
        pmse = np.zeros([5, 1], dtype=float)
        pmae = np.zeros([5, 1], dtype=float)
        poly = PolynomialFeatures(degree = int(m))
        #Split
        for train_index, val_index in kf.split(x_train, y_train):
            data_train, data_val = x_train[train_index], x_train[val_index]
            label_train, label_val = y_train[train_index], y_train[val_index]
            #Poly transform
            data_train_expand = poly.fit_transform(data_train)
            data_val_expand = poly.fit_transform(data_val)
            #Define the model, fit and predict 
            reg = LinearRegression().fit(data_train_expand, label_train)
            y_pred = reg.predict(data_val_expand)
            pmse[k] = p_MSE(label_val, y_pred)
            pmae[k] = p_MAE(label_val, y_pred)
            k += 1
        #Store the values
        pmse_av = np.mean(pmse)
        pmae_av = np.mean(pmae)
        pMSE[Nm] = pmse_av
        pMAE[Nm] = pmae_av
    return pMSE, pMAE



#without cross validation 
def polyVal(x_train, y_train,x_val,y_val, min_M, max_M):
    
    poly_order = np.linspace(min_M, max_M, num = max_M - min_M+1)
    pMSE = np.zeros([max_M - min_M +1, 1], dtype=float)
    pMAE = np.zeros([max_M - min_M +1, 1], dtype=float)
    
    for Nm, m in enumerate(poly_order):
        #poly feature transform
        poly = PolynomialFeatures(degree = int(m))
        data_train_expand = poly.fit_transform(x_train)
        data_val_expand = poly.fit_transform(x_val)
        #Define the model, fit and predict 
        reg = LinearRegression().fit(data_train_expand, y_train)
        y_pred = reg.predict(data_val_expand)
        
        pMSE[Nm] = p_MSE(y_val,y_pred)
        pMAE[Nm] = p_MAE(y_val,y_pred)
    return pMSE, pMAE



#Ploy degree from 2 to 3 
pMSE, pMAE = polyVal(X_train, Y_train,X_val,Y_val, 2, 3)
print('Non-Linear Regression with PolyFeature(2-3)')
print('pMSE:\n', pMSE)
print('pMAE:\n', pMAE)


Non-Linear Regression with PolyFeature(2-3)
pMSE:
 [[1.41582926e+22]
 [9.59203000e+02]]
pMAE:
 [[7.32488935e+09]
 [9.48216817e+00]]


In [35]:
# The Above result is too bad, we try poly order 4 with PCA feature Reduction 

#Using PCA to reduce features to 20

#PCA
pca = PCA(n_components = 20).fit(X_train)
X_train_pca = pca.transform(X_train)
X_val_pca = pca.transform(X_val)

#PolyFeature Transform with degree=4
poly = PolynomialFeatures(degree = 4)
data_train_expand = poly.fit_transform(X_train_pca)
data_val_expand = poly.fit_transform(X_val_pca)
reg = LinearRegression().fit(data_train_expand, Y_train)
Y_pred = reg.predict(data_val_expand)

#Calulate PMSE and PMAE
pMSE = p_MSE(Y_val,Y_pred)
pMAE = p_MAE(Y_val,Y_pred)
print('Non-Linear Regression with PolyFeature 4')
print('pMSE:',pMSE)
print('pMAE:',pMAE)


Non-Linear Regression with PolyFeature 4
pMSE: 689376.4219384738
pMAE: 61.591328662149465


### 4.2.1 Non-Linear Regression with PolyFeatures Result:
#### We found the best poly order is 3 
#### Validation Results:
#### pMSE: 3.08009484e+04
#### pMAE: 1.08889408e+01

### 4.2.2 Report Best Non-Linear Regression Model(Entire Training Dataset)

In [ ]:
#PolyFeature Transform with degree=3
poly_best = PolynomialFeatures(degree = 3)
data_train_expand = poly_best.fit_transform(X_train_all)
data_val_expand = poly_best.fit_transform(X_val_all)

#Regression model
reg_best = LinearRegression().fit(data_train_expand, Y_train_all)
Y_pred_all = reg_best.predict(data_val_expand)

#Performance
MAE = mean_absolute_error(Y_val_all,Y_pred_all)
pMSE = p_MSE(Y_val_all,Y_pred_all)
pMAE = p_MAE(Y_val_all,Y_pred_all)
rSquared = R_squared(Y_val_all,Y_pred_all)
m_rSquared = modified_R_squared(Y_val_all, Y_pred_all, pMSE)
print('Best Non-Linear Regression(PolyFeature=3)')
print('MAE:',MAE)
print('pMSE:',pMSE)
print('pMAE:',pMAE)
print('R_squared:', rSquared)
print('Modified_R_Squared:',m_rSquared)

## 4.3 Ridge Regression 

In [138]:
#Ridge Regression with 5 fold cross validation 
def RidgeCrossValidation(x_train, y_train, min_M, max_M):
    kf = KFold(n_splits=5)
    poly_order = np.linspace(min_M, max_M, num = max_M - min_M+1)
    pMSE = np.zeros([max_M - min_M +1, 50], dtype=float)
    pMAE = np.zeros([max_M - min_M +1, 50], dtype=float)

    alphas = np.logspace(-3, 3, 50, base=10)
    
    for Nm, m in enumerate(poly_order):
        for Na, a in enumerate(alphas):
            
            k = 0
            pmse_av = 0
            pmae_av = 0
            pmse = np.zeros([5, 1], dtype=float)
            pmae = np.zeros([5, 1], dtype=float)
            poly = PolynomialFeatures(degree = int(m))
            for train_index, val_index in kf.split(x_train, y_train):
                data_train, data_val = x_train[train_index], x_train[val_index]
                label_train, label_val = y_train[train_index], y_train[val_index]
                #Poly fit transformation
                data_train_expand = poly.fit_transform(data_train)
                data_val_expand = poly.fit_transform(data_val)
                #Ridge Regression fit and predict
                ridge = Ridge(alpha = a)
                ridge.fit(data_train_expand, label_train)
                y_pred = ridge.predict(data_val_expand)
                pmse[k] = p_MSE(label_val, y_pred)
                pmae[k] = p_MAE(label_val, y_pred)
                k += 1
            pmse_av = np.mean(pmse)
            pmae_av = np.mean(pmae)
            pMSE[Nm,Na] = pmse_av
            pMAE[Nm,Na] = pmae_av
    return pMSE, pMAE

# Ridge Without cross validation 
def RidgeVal(x_train, y_train,x_val,y_val, min_M, max_M):
    
    poly_order = np.linspace(min_M, max_M, num = max_M - min_M+1)
    pMSE = np.zeros([max_M - min_M +1, 50], dtype=float)
    pMAE = np.zeros([max_M - min_M +1, 50], dtype=float)

    alphas = np.logspace(0, 6, 50, base=10)
    
    for Nm, m in enumerate(poly_order):
        
        poly = PolynomialFeatures(degree = int(m))
        #Poly fit transformation
        data_train_expand = poly.fit_transform(x_train)
        data_val_expand = poly.fit_transform(x_val)
        
        for Na, a in enumerate(alphas):
        
            #Ridge Regression fit and predict
            ridge = Ridge(alpha = a)
            ridge.fit(data_train_expand, y_train)
            y_pred = ridge.predict(data_val_expand)
                
            pMSE[Nm,Na] = p_MSE(y_val, y_pred)
            pMAE[Nm,Na] = p_MAE(y_val, y_pred)
    return pMSE, pMAE


#Ploy degree from 1 to 2
pMSE, pMAE = RidgeVal(X_train, Y_train, X_val, Y_val, 1, 2)
print('Ridge Regression with Poly degree 1-2::')
print('pMSE:\n', pMSE)
print('pMAE:\n', pMAE)

Ridge Regression with Poly degree 1-2::
pMSE:
 [[ 11.52411335  11.53149948  11.54073863  11.55207274  11.56562143
   11.58127119  11.59854507  11.61648978  11.63364174  11.64813896
   11.65801336  11.66162872  11.65815064  11.64788314  11.63229967
   11.61364237  11.59407609  11.57457895  11.5539792   11.52866309
   11.49331727  11.44263918  11.37348957  11.28677616  11.18853431
   11.09002824  11.0069645   10.95795533  10.9622788   11.03697192
   11.1935275   11.43488726  11.75374554  12.13305427  12.54891956
   12.97509095  13.38756724  13.76792394  14.10469924  14.3930255
   14.63319279  14.8288784   14.9855493   15.10926666  15.20591784
   15.28079918  15.33844433  15.38260392  15.41630659  15.44195545]
 [238.10646022 229.36832661 219.11647597 207.48210515 194.71806842
  181.17648464 167.26116504 153.37452203 139.87697752 127.06419235
  115.15560679 104.28650708  94.50318875  85.76687962  77.97062546
   70.96703351  64.59991504  58.73225015  53.26507138  48.14470911
   43.35871871 

### 4.3.1 Find the best poly order M and alpha for Ridge Regrssion 

In [ ]:
#The best pmse is happended in M =1
best_pmse_list = pMSE[0]

#Find the best M and Alpha based on the best pmse list which is Poly Order 1
best_pmse = np.min(best_pmse_list)
best_index = np.where(best_pmse_list == best_pmse)
alphas = np.logspace(0, 6, 50, base=10)
best_alpha = alphas[best_index] 
print('Best Alpha is:', best_alpha)  

### 4.3.2 Report the Validation Result  For Ridge Regression(Entire Training data)
### under the Best M and alpha:
#### Best M = 1
#### Best alpha = 2023.58964773

In [8]:
poly = PolynomialFeatures(degree = 1)
#Poly fit transformation
X_train_expand = poly.fit_transform(X_train_all)
X_val_expand = poly.fit_transform(X_val_all)

ridge = Ridge(alpha = 2023.58964773)
ridge.fit(X_train_expand, Y_train_all)
Y_pred_all = ridge.predict(X_val_expand)

#Performance Measurement 
MAE = mean_absolute_error(Y_val_all,Y_pred_all)
pMSE = p_MSE(Y_val_all,Y_pred_all)
pMAE = p_MAE(Y_val_all,Y_pred_all)
rSquared = R_squared(Y_val_all,Y_pred_all)
m_rSquared = modified_R_squared(Y_val_all, Y_pred_all, pMSE)
print('Best Ridge Regression Model(On Entire Data Set)\n(poly order M=1, alpha = 2023.58964773)')
print('MAE:',MAE)
print('pMSE:',pMSE)
print('pMAE:',pMAE)
print('R_squared:', rSquared)
print('Modified_R_Squared:',m_rSquared)

Best Ridge Regression Model(On Entire Data Set)
(poly order M=1, alpha = 2023.58964773)
MAE: 2980.8640797484436
pMSE: 8.810311914869368
pMAE: 1.6891806743341236
R_squared: 0.026793606627427913
Modified_R_Squared: -0.004450446693054166


### 4.4 KNN
#### Since KNN runs fast, we will use the whole data set for Training and Validation 

In [21]:
#KNN with cross validation 
def KNNCrossValidation(x_train, y_train):
    kf = KFold(n_splits=5)
    K_ = np.linspace(5,15,num=11)
    pMSE = np.zeros([11, 1], dtype=float)
    pMAE = np.zeros([11, 1], dtype=float)
    
    for Nk, k in enumerate(K_):
        j = 0
        pmse_av = 0
        pmae_av = 0
        pmse = np.zeros([5, 1], dtype=float)
        pmae = np.zeros([5, 1], dtype=float)
        neigh = KNeighborsRegressor(n_neighbors=int(k))
        for train_index, val_index in kf.split(x_train, y_train):
            data_train, data_val = x_train[train_index], x_train[val_index]
            label_train, label_val = y_train[train_index], y_train[val_index]
            
            neigh.fit(data_train, label_train)
            y_pred = neigh.predict(data_val)
            pmse[j] = p_MSE(label_val, y_pred)
            pmae[j] = p_MAE(label_val, y_pred)
            j += 1
        pmse_av = np.mean(pmse)
        pmae_av = np.mean(pmae)
        pMSE[Nk] = pmse_av
        pMAE[Nk] = pmae_av
    return pMSE, pMAE


#Without Cross Validation 
def KNNVal(x_train, y_train, x_val, y_val):
    
    K_ = np.linspace(5,20,num=16)
    pMSE = np.zeros([16, 1], dtype=float)
    pMAE = np.zeros([16, 1], dtype=float)
    
    for Nk, k in enumerate(K_):
        #Define the model, fit and predict 
        neigh = KNeighborsRegressor(n_neighbors=int(k))
        neigh.fit(x_train, y_train)
        y_pred = neigh.predict(x_val)

        pMSE[Nk] = p_MSE(y_val, y_pred)
        pMAE[Nk] = p_MAE(y_val, y_pred)
    return pMSE, pMAE

#KNN Regression
pMSE, pMAE = KNNVal(X_train_all, Y_train_all, X_val_all, Y_val_all)
print('KNN Regression(Entire Data Set):')
print('pMSE:\n', pMSE)
print('pMAE:\n', pMAE)

KNN Regression(Entire Data Set):
pMSE:
 [[14.98001292]
 [13.02720829]
 [11.5422054 ]
 [10.90776939]
 [10.57448586]
 [10.96239722]
 [10.2111965 ]
 [10.16886223]
 [ 9.65904242]
 [ 9.34504953]
 [ 8.95708318]
 [ 8.93919191]
 [ 9.17997126]
 [ 9.19342601]
 [ 9.02648365]
 [ 9.15755031]]
pMAE:
 [[1.49429489]
 [1.4620734 ]
 [1.45508871]
 [1.46410538]
 [1.46849502]
 [1.46912377]
 [1.45976684]
 [1.45408917]
 [1.44972021]
 [1.44969397]
 [1.44632021]
 [1.44688108]
 [1.45547524]
 [1.46207498]
 [1.45855727]
 [1.45426138]]


### 4.4.1 KNN Model Parameters and Validation Result

In [15]:
#Find Best K 
K_ = np.linspace(5,20,num=16)
best_K_index = np.where(pMSE == np.min(pMSE))
best_K = K_[best_K_index[0]]
print('KNN Regression Model:')
print('Best K for KNN:', best_K[0])
print('pMSE:', np.min(pMSE))
print('pMAE:', np.min(pMAE))

KNN Regression Model:
Best K for KNN: 5.0
pMSE: 1.1389524902128147
pMAE: 0.5698028364365723


### 4.4.2 Report Best KNN Model Validation Result (Entire Dataset)

In [17]:
knn_best = KNeighborsRegressor(n_neighbors=16)
knn_best.fit(X_train_all, Y_train_all)
Y_pred_all = knn_best.predict(X_val_all)

#Performance Measurement 
MAE = mean_absolute_error(Y_val_all,Y_pred_all)
pMSE = p_MSE(Y_val_all,Y_pred_all)
pMAE = p_MAE(Y_val_all,Y_pred_all)
rSquared = R_squared(Y_val_all,Y_pred_all)
m_rSquared = modified_R_squared(Y_val_all, Y_pred_all, pMSE)
print('Best KNN Model(On Entire Data Set)\n(K=16)')
print('MAE:',MAE)
print('pMSE:',pMSE)
print('pMAE:',pMAE)
print('R_squared:', rSquared)
print('Modified_R_Squared:',m_rSquared)

Best KNN Model(On Entire Data Set)
(K=16)
MAE: 2908.810886870529
pMSE: 8.939191908350137
pMAE: 1.4468810825132776
R_squared: -0.006969376991294185
Modified_R_Squared: -0.019143861440728882


## 5. Finalize the best Model
#### We choose SVR with [g,C] = [gamma_[7], C_[40]] = [0.00719686, 79.06043211]

In [77]:
# Run best model with test set

In [18]:

#Load data
train_data = df_orig_train_data.to_numpy()
train_label = df_orig_label_data.to_numpy()
train_label = train_label[:,0]

X_test = X_orig_test.to_numpy()
Y_test = Y_orig_test.to_numpy()


#Normalize Training and Test data
scaler_best = StandardScaler()
scaler_best.fit(train_data)
train_data = scaler_best.transform(train_data)
test_data = scaler_best.transform(X_test)

#Fit and Predict model 

#Gamma and C
gamma_ = np.logspace(-3, 3, num=50, base=10.0)
C_ = np.logspace(-3, 3, num=50, base=10.0)

#Best C is the 40th C, best gamma is 7th
model = SVR(C=C_[40], kernel='rbf', gamma=gamma_[7])
model.fit(train_data, train_label)
test_pred = model.predict(test_data)

#Performance Measure
pMSE = p_MSE(Y_test,test_pred)
pMAE = p_MAE(Y_test,test_pred)
rSquared = R_squared(Y_test,test_pred)
m_rSquared = modified_R_squared(Y_test, test_pred, pMSE)
print('Best Model(SVR)')
print('pMSE:',pMSE)
print('pMAE:',pMAE)
print('R_squared:', rSquared)
print('Modified_R_Squared:',m_rSquared)


Best Model(SVR)
pMSE: 1.9488751498926589
pMAE: 0.7151657983768334
R_squared: [-0.03669242]
Modified_R_Squared: [0.77620322]


# 6. Save .pkl File

In [19]:
#Pipe Line
pipe = Pipeline([('scaler', scaler_best), ('svr', model)])
test_pred = pipe.predict(X_test)

pMSE = p_MSE(Y_test,test_pred)
pMAE = p_MAE(Y_test,test_pred)
rSquared = R_squared(Y_test,test_pred)
m_rSquared = modified_R_squared(Y_test, test_pred, pMSE)
print('Best Model(SVR)')
print('pMSE:',pMSE)
print('pMAE:',pMAE)
print('R_squared:', rSquared)
print('Modified_R_Squared:',m_rSquared)


Best Model(SVR)
pMSE: 1.9488751498926589
pMAE: 0.7151657983768334
R_squared: [-0.03669242]
Modified_R_Squared: [0.77620322]


In [20]:
#Save Best Model
alias = "NEWS"
filename = "model_{}.pkl".format(alias)
pickle.dump(pipe, open(filename, 'wb'))
print("A model has been saved as {}".format(filename))

A model has been saved as model_NEWS.pkl
